In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import rdkit


/auto/home/menuab/miniforge3/envs/titan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
tokenizer  = AutoTokenizer.from_pretrained("/auto/home/menuab/code/YNNtitan/torchtitan/tokenizers/Llama-3.2-chem-1B-v1")
model = AutoModelForCausalLM.from_pretrained("/nfs/h100/raid/chem/checkpoints/hf/yerevann/Llama-3.2-1B_conformers/9a8a8bacd6b542c09f5df78b/step-13000").to("cuda:1")

In [43]:
model.dtype, model.device

(torch.float32, device(type='cuda', index=1))

In [44]:
canonical = rdkit.Chem.MolToSmiles(rdkit.Chem.MolFromSmiles("CC(C)(C#N)C(O)=NCCC1CCCO1"), canonical=True, isomericSmiles=False)
prompt = f"[SMILES]{canonical}[/SMILES]"
# prompt = "[SMILES]CC(C)(C#N)C(O)=NCCC1CCCO1[/SMILES]"
prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).to(model.device).input_ids
prompt

tensor([[128000, 128256,   3791,   3100,   2432,     34,      2,     45,      8,
             34,  19792,  11992,     45,  54973,     16,   3791,   8445,     16,
         128257]], device='cuda:1')

In [45]:
tokenizer.batch_decode(prompt)

['<|begin_of_text|>[SMILES]CC(C)(C#N)C(O)=NCCC1CCCO1[/SMILES]']

In [46]:
output = model.generate(prompt, max_length=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [47]:
tokenizer.batch_decode(output)

['<|begin_of_text|>[SMILES]CC(C)(C#N)C(O)=NCCC1CCCO1[/SMILES][PROPERTY]6.63[/PROPERTY][CONFORMER]C<0.9923,-0.0193,0.0773>C<2.5291,-0.0394,0.0309>(C<3.0443,-1.4709,0.3538>)(C<3.0151,0.9011,1.1011>#N<3.3751,']